# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
from train import clean_data
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from pprint import pprint
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import Model
import requests
import json

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'wine-quality-automl'

experiment=Experiment(ws, experiment_name)

# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "wine-quality"
description_text = "Wine Quality Dataset for Udacity Course 3"

if key in ws.datasets.keys(): 
    found = True
    input_data = ws.datasets[key] 

if not found:
    # Create AML Dataset and register it into Workspace
    url_white = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv"
    url_red = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    white_data = TabularDatasetFactory.from_delimited_files(url_white, separator=";")
    red_data = TabularDatasetFactory.from_delimited_files(url_red, separator=";")
    features, target = clean_data(white_data, red_data)
    features.loc[:,"quality"]=target
    ds = ws.get_default_datastore()
    input_data = TabularDatasetFactory.register_pandas_dataframe(dataframe=features, target=ds, name=key,
                                                             description=description_text)


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/8d4f60a0-5c1c-4cd2-96e9-0be9002ae3ea/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [3]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "aml-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2',# for GPU, use "STANDARD_NC6"
                                                           vm_priority = 'lowpriority', # optional
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Creating
Succeeded............................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
automl_settings = {
    "n_cross_validations": 3,
    "enable_early_stopping": True,
    "primary_metric": "AUC_weighted",
    "max_concurrent_iterations": 4,
    
}
automl_config = AutoMLConfig(
    experiment_timeout_hours=1.0,
    task="classification",
    compute_target=compute_target,
    training_data=input_data,
    label_column_name="quality",
    **automl_settings)

In [5]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config, tag={'type':'automl'})

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=====

        61    StackEnsemble                                 0:02:06       0.8675    0.8711


{'runId': 'AutoML_0bd5b3c9-103e-49c0-93e2-288080a48f91',
 'target': 'aml-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-01-15T09:08:59.508063Z',
 'endTimeUtc': '2021-01-15T09:38:50.551587Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'aml-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"wine-quality-automl","subscription_id":"6b4af8be-9931-443e-90f6-c4c34a1f9737","resource_group":"aml-quickstarts-134739","workspace_name":"quick-starts-ws-134739","region":"southcentralus","compute_target":"aml-compute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_cl

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [7]:
assert (automl_run.get_status() == "Completed")
best_ml_run, best_model = automl_run.get_output()


def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators':
                       list(e[0] for e in step[1].estimators),
                   'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()
            
print_model(best_model)


datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['41', '32', '33', '34', '36', '58', '46'],
 'weights': [0.4666666666666667,
             0.06666666666666667,
             0.13333333333333333,
             0.06666666666666667,
             0.06666666666666667,
             0.06666666666666667,
             0.13333333333333333]}

41 - standardscalerwrapper
{'class_name': 'StandardScaler',
 'copy': True,
 'module_name': 'sklearn.preprocessing._data',
 'with_mean': False,
 'with_std': False}

41 - xgboostclassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'eta': 0.5,
 'gamma': 0.1,
 'learning_ra

In [8]:
best_ml_run.download_file("./outputs/model.pkl")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [30]:
registered_model = best_ml_run.register_model(model_name="wine-quality-model", model_path="./outputs/model.pkl")
myenv = Environment(name="myenv")
conda_dep = CondaDependencies()
for conda_pkg in ["scikit-learn", "py-xgboost","numpy", "pip"]:
    conda_dep.add_conda_package(conda_pkg)

for pip_pkg in ["azureml-defaults","azureml-automl-core","azureml-automl-runtime","packaging", "inference-schema[numpy-support,pandas-support]"]:
    conda_dep.add_pip_package(pip_pkg)


with open("env.yml","w") as f:
    f.write(conda_dep.serialize_to_string())

myenv.python.conda_dependencies = conda_dep

service_name = 'wine-quality-classification'
inference_config = InferenceConfig(entry_script='score.py', environment=myenv)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True, enable_app_insights=True)
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)
print(service.scoring_uri)
primary_key, _ = service.get_keys()

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.......................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://4943e30c-57e0-4af3-b536-36791bb25211.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [32]:
data = {"data":
         [{
             "fixed acidity":7.90000,
             "volatile acidity":0.52000,
             "citric acid":0.26000, 
             "residual sugar":2.2000,
             "chlorides":0.07900,
             "free sulfur dioxide":14.0000,
             "total sulfur dioxide":38.0000,
             "density":0.99675,
             "pH":3.31000,
             "sulphates":0.62000,
             "alcohol":10.20000,
             "is_red_wine": 1
         },
         {
             "fixed acidity":6.80,
             "volatile acidity":0.26000,
             "citric acid":0.32000, 
             "residual sugar":5.2000,
             "chlorides":0.04300,
             "free sulfur dioxide":34.0000,
             "total sulfur dioxide":134.00000,
             "density":0.99374,
             "pH":3.18000,
             "sulphates":0.47000,
             "alcohol":10.40000,
             "is_red_wine": 0
         }]
     }
header = {'Content-Type': 'application/json'}
header["Authorization"] = f"Bearer {primary_key}"
json_data = json.dumps(data)

resp = requests.post(service.scoring_uri, json_data, headers=header)
print(resp.text)

list index out of range


TODO: In the cell below, print the logs of the web service and delete the service

In [33]:
print(service.get_logs())
#service.delete()

2021-01-15T11:49:52,376213183+00:00 - gunicorn/run 
2021-01-15T11:49:52,376266283+00:00 - iot-server/run 
2021-01-15T11:49:52,382760396+00:00 - rsyslog/run 
2021-01-15T11:49:52,387298006+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_c1774ef3adee538541c4a64455d2905d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c1774ef3adee538541c4a64455d2905d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c1774ef3adee538541c4a64455d2905d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c1774ef3adee538541c4a64455d2905d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c1774ef3adee538541c4a64455d2905d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC